This project is make do detect which tweets is real about disaster and which one is fake. tthis is classification peoblem containing two categories i.e fake or real. tthis project is created using RNN with LSTM and also contain two dense layers. Dataset is taken from kaggle. 

In [0]:
import numpy as np 
import pandas as pd 
%tensorflow_version 2.x
import tensorflow as tf

In [4]:
data=pd.read_csv("train.csv")
data.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [5]:
data.shape

(7613, 5)

In [0]:
data.drop(["location", "keyword"], axis=1, inplace=True)

In [9]:
labels=data["target"]
labels.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [0]:
data.drop("target", axis=1, inplace=True)

In [11]:
data.head()

,id,text
0,1,Our Deeds are the Reason of this #earthquake M...
1,4,Forest fire near La Ronge Sask. Canada
2,5,All residents asked to 'shelter in place' are ...
3,6,"13,000 people receive #wildfires evacuation or..."
4,7,Just got sent this photo from Ruby #Alaska as ...


In [12]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stoplist = stopwords.words('english')

training_data = data.text.tolist()
train_data=[]

for i in training_data:
    string=""
    for j in i.lower().split():
        if j not in stoplist: #removeing stopwords from tweets
            string= string+j+" "
            
    train_data.append(string.rstrip())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
train_data[0:5]

['deeds reason #earthquake may allah forgive us',
 'forest fire near la ronge sask. canada',
 "residents asked 'shelter place' notified officers. evacuation shelter place orders expected",
 '13,000 people receive #wildfires evacuation orders california',
 'got sent photo ruby #alaska smoke #wildfires pours school']

In [15]:
training_lables=np.array(labels.to_numpy())
training_lables[:5]

array([1, 1, 1, 1, 1])

In [17]:
# checking how long a tweet is present in all reviews

review_length=[len(i.split()) for i in training_data]
print(max(review_length))

31


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
vocab_size=10000
OOV_tok="<OOV>"
padding_type="post"
trunc_type="post"
max_length=25

tokenizer=Tokenizer(num_words=vocab_size, oov_token=OOV_tok)
tokenizer.fit_on_texts(training_data)
word_index=tokenizer.word_index
#print(word_index)

training_sequences=tokenizer.texts_to_sequences(training_data)
training_pad_seq=pad_sequences(training_sequences,padding=padding_type, maxlen=max_length, truncating=trunc_type)

In [0]:
# reverse the word_index dictionary

reverse_words=dict([(j, i) for i,j in tokenizer.word_index.items()])
#print(reverse_words)

In [21]:
def decode_review(review):
    return " ".join([reverse_words.get(i, "?") for i in review])
    
print(decode_review(training_pad_seq[1]))
print(training_data[1])

forest fire near la ronge sask canada ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Forest fire near La Ronge Sask. Canada


In [0]:
emb_dim=16

model=tf.keras.models.Sequential([
                                tf.keras.layers.Embedding(vocab_size, emb_dim, input_length=max_length),
                                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                                tf.keras.layers.Dense(6, activation="relu"),
                                tf.keras.layers.Dense(1, activation="sigmoid")
])

In [23]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 16)            160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               41472     
_________________________________________________________________
dense (Dense)                (None, 6)                 774       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 202,253
Trainable params: 202,253
Non-trainable params: 0
_________________________________________________________________


In [24]:
epochs=10

history=model.fit(training_pad_seq, training_lables, epochs=epochs)

Train on 7613 samples
Epoch 1/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.5373 - accuracy: 0.7218
Epoch 2/10
7613/7613 [==============================] - 2s 320us/sample - loss: 0.3410 - accuracy: 0.8588
Epoch 3/10
7613/7613 [==============================] - 2s 307us/sample - loss: 0.2579 - accuracy: 0.9029
Epoch 4/10
7613/7613 [==============================] - 2s 313us/sample - loss: 0.2050 - accuracy: 0.9258
Epoch 5/10
7613/7613 [==============================] - 2s 313us/sample - loss: 0.1697 - accuracy: 0.9421
Epoch 6/10
7613/7613 [==============================] - 2s 311us/sample - loss: 0.1434 - accuracy: 0.9511
Epoch 7/10
7613/7613 [==============================] - 2s 312us/sample - loss: 0.1273 - accuracy: 0.9585
Epoch 8/10
7613/7613 [==============================] - 2s 311us/sample - loss: 0.1144 - accuracy: 0.9631
Epoch 9/10
7613/7613 [==============================] - 2s 312us/sample - loss: 0.0957 - accuracy: 0.9679
Epoch 10/10
7613/7613 [==

In [25]:
# using test data for testing

test_data=pd.read_csv("test.csv")
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [26]:
testing_data=test_data.text.tolist()
testing_data[:3]

['Just happened a terrible car crash',
 'Heard about #earthquake is different cities, stay safe everyone.',
 'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all']

In [0]:
testing_sequences=tokenizer.texts_to_sequences(testing_data)
testing_pad_sequence=pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [0]:
# making predictions on test data

test_predictions=model.predict_classes(testing_pad_sequence)

In [30]:
test_predictions[:10]

array([[0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0]], dtype=int32)

After submitting these predictions on kaggle i got 78% accuracy.